In [2]:
!pip install torch transformers peft bitsandbytes accelerate datasets huggingface_hub pymupdf pandas docx
!pip install ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.5 MB/s eta 0:00:00
   

In [3]:
!pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.5 MB/s eta 0:00:00


In [4]:
!sudo apt-get update
!sudo apt-get install libstdc++6

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,731 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,774 kB]
Get:13 http://archive.ubuntu.com

In [5]:
!pip install --upgrade bitsandbytes


In [6]:
import os
import json
import fitz  # PyMuPDF
import pandas as pd
from docx import Document
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, pipeline
from peft import LoraConfig, get_peft_model, TaskType
import torch

In [ ]:
# Load Qwen model and tokenizer
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [9]:
# ✅ Extract Text Functions
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using PyMuPDF (fitz)."""
    text_data = []
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text = page.get_text("text")
            if text:
                text_data.append(text.strip())
    except Exception as e:
        print(f"Error reading PDF: {e}")
    return text_data

def extract_text_from_docx(docx_path):
    """Extract text from a Word Document (.docx)."""
    text_data = []
    doc = Document(docx_path)
    for para in doc.paragraphs:
        if para.text.strip():
            text_data.append(para.text.strip())
    return text_data

def extract_text_from_txt(txt_path):
    """Extract text from a TXT file."""
    with open(txt_path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f.readlines() if line.strip()]

def extract_text_from_csv_or_excel(file_path):
    """Extract Q&A text from CSV or Excel files."""
    try:
        df = pd.read_csv(file_path) if file_path.endswith(".csv") else pd.read_excel(file_path)
        json_entries = []
        for _, row in df.iterrows():
            entry = {
                "instruction": str(row.get("Question", "Summarize the following text:")),
                "input": str(row.get("Context", row.to_string())),
                "output": str(row.get("Answer", "Generated summary..."))
            }
            json_entries.append(entry)
        return json_entries
    except Exception as e:
        print(f"Error reading CSV/Excel: {e}")
        return []

# ✅ Convert Text to JSONL
def convert_to_jsonl(file_path, output_jsonl="train_data.jsonl"):
    """Detect file type, extract text, and save as JSONL."""
    file_ext = os.path.splitext(file_path)[1].lower()
    extracted_texts = []

    if file_ext == ".pdf":
        extracted_texts = extract_text_from_pdf(file_path)
    elif file_ext == ".docx":
        extracted_texts = extract_text_from_docx(file_path)
    elif file_ext == ".txt":
        extracted_texts = extract_text_from_txt(file_path)
    elif file_ext in [".csv", ".xlsx"]:
        extracted_data = extract_text_from_csv_or_excel(file_path)
        if extracted_data:
            with open(output_jsonl, "w", encoding="utf-8") as f:
                for entry in extracted_data:
                    f.write(json.dumps(entry, ensure_ascii=False) + "\n")
            return

    json_entries = [{"instruction": "Summarize the following text:", "input": text, "output": "Generated summary..."} for text in extracted_texts]

    with open(output_jsonl, "w", encoding="utf-8") as f:
        for entry in json_entries:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")


In [10]:
from google.colab import files
uploaded = files.upload()


Saving 2501.12948v1.pdf to 2501.12948v1.pdf
Saving dataset.md to dataset.md
Saving deepseekv3-cost-explained.md to deepseekv3-cost-explained.md
Saving deepseekv3-explained.md to deepseekv3-explained.md
Saving design-notes-3fs.md to design-notes-3fs.md
Saving open-source-week.md to open-source-week.md


In [11]:
# Update the folder path where your files are located after upload
folder_path = '/content'  # Colab default path

# Define paths to the files
pdf_file = os.path.join(folder_path, '2501.12948v1.pdf')
md_files = [os.path.join(folder_path, 'dataset.md'),
            os.path.join(folder_path, 'deepseekv3-cost-explained.md'),
            os.path.join(folder_path, 'deepseekv3-explained.md'),
            os.path.join(folder_path, 'design-notes-3fs.md'),
            os.path.join(folder_path, 'open-source-week.md')]

In [ ]:
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF using PyMuPDF (fitz)"""
    text_data = []
    doc = fitz.open(pdf_path)
    for page in doc:
        text = page.get_text("text")
        if text:
            text_data.append(text.strip())
    return text_data

# Function to extract text from a markdown file
def extract_text_from_md(md_path):
    """Extract text from Markdown files (.md)"""
    with open(md_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

# Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_file)

# Extract text from Markdown files
md_texts = []
for md_file in md_files:
    md_texts.append(extract_text_from_md(md_file))

# Combine PDF and Markdown texts
all_texts = pdf_text + md_texts  # You can adjust how you want to combine them.

# Convert the extracted texts into JSONL format
def convert_to_jsonl(texts, output_jsonl="train_data.jsonl"):
    """Convert extracted text to JSONL format"""
    json_entries = [{"instruction": "Summarize the following text:", "input": text, "output": "Generated summary..."} for text in texts]

    # Save as JSONL
    with open(output_jsonl, "w", encoding="utf-8") as f:
        for entry in json_entries:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

# Call the function to save the JSONL
convert_to_jsonl(all_texts)

In [13]:
# Load the dataset
dataset = load_dataset("json", data_files={"train": "train_data.jsonl"})
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 5
    })
})


In [14]:
# ✅ Tokenization
def tokenize_function(examples):
    inputs = [q + " " + a for q, a in zip(examples["instruction"], examples["output"])]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = model_inputs["input_ids"]
    return model_inputs

dataset = dataset.map(tokenize_function, batched=True)
train_test_split = dataset["train"].train_test_split(test_size=0.1)


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [15]:
# ✅ LoRA Configuration
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, config)

In [16]:
# ✅ Training Arguments
training_args = TrainingArguments(
    output_dir="./fine-tuned-qwen",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=1e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none",
    optim="adamw_torch"
)

In [17]:
# ✅ Train Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_split["train"],
    eval_dataset=train_test_split["test"],
)
trainer.train()
trainer.save_model()
torch.cuda.empty_cache()

Step,Training Loss


In [18]:
# ✅ Convert to GGUF
os.system("python convert-hf-to-gguf.py --model ./fine-tuned-qwen --output fine_tuned_qwen.gguf")


512

In [ ]:
# ✅ Inference Pipeline
qa_pipeline = pipeline("text-generation", model="./fine-tuned-qwen", tokenizer=tokenizer)
question = "What is a Transformer model?"
response = qa_pipeline(question, max_length=200)
print(response)

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': "What is a Transformer model? A transformer model is a type of deep learning architecture that has been developed by Google's researchers to improve the performance of machine translation, natural language processing (NLP) tasks and other NLP applications. The key innovation in"}]
